In [1]:
!pip install tensorflow-gpu

In [2]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [3]:
device_name = tf.test.gpu_device_name()
if device_name!='/device:GPU:0':
    raise SystemError('GPU deivce not found')
print('Found GPU at:{}'.format(device_name))

Found GPU at:/device:GPU:0


In [4]:
from google.colab import files
file = files.upload()

In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import pickle

import numpy as np

Using TensorFlow backend.


In [6]:
K.set_learning_phase(1)

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(50, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
    
for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)

/home/quang/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [7]:
# def preprocess_input(x):
#     # Change BRG => RGB
#     x = x[:, :, :, ::-1]

#     # Normalize
#     x /= 255.
#     x -= 0.5
#     x *= 2.
#     return x

def _imagenet_preprocess_input(x):
    """
    For ResNet50, VGG models. For InceptionV3 and Xception it's okay to use the
    keras version (e.g. InceptionV3.preprocess_input) as the code path they hit
    works okay with tf.Tensor inputs. The following was translated to tf ops from
    https://github.com/fchollet/keras/blob/fb4a0849cf4dc2965af86510f02ec46abab1a6a4/keras/applications/imagenet_utils.py#L52
    It's a possibility to change the implementation in keras to look like the
    following and modified to work with BGR images (standard in Spark), but not doing it for now.
    """
    # assuming 'BGR'
    # Zero-center by mean pixel
    mean = np.ones((256,256,3), dtype=np.float32)
    mean[..., 0] = 103.939
    mean[..., 1] = 116.779
    mean[..., 2] = 123.68
    return x - mean 

x_train = pickle.load(open("x_train.pickle","rb"))
y_train = pickle.load(open("y_train.pickle","rb"))

BATCH_SIZE = 8
EPOCHS = 30
# x_train = preprocess_input(x_train)
x_train = _imagenet_preprocess_input(x_train)
print ()

In [10]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam',metrics=['accuracy'], loss='categorical_crossentropy')
EPOCHS = 40
# train the model on the new data for a few epochs
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")
model.fit_generator(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) // BATCH_SIZE,
                    epochs=EPOCHS)

for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0005, momentum=0.9),metrics=['accuracy'], loss='categorical_crossentropy')
# model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')
EPOCHS = 50
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
aug = ImageDataGenerator(rotation_range=5,
                         zoom_range=0.1,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.1,
                         horizontal_flip=False,
                         fill_mode="nearest")
model.fit_generator(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) // BATCH_SIZE,
                    epochs=EPOCHS)
print ()


Epoch 1/40
6/6 [==============================] - 1s 245ms/step - loss: 6.2144 - accuracy: 0.0000e+00
Epoch 2/40
6/6 [==============================] - 1s 114ms/step - loss: 4.5917 - accuracy: 0.0417
Epoch 3/40
6/6 [==============================] - 1s 103ms/step - loss: 4.1213 - accuracy: 0.0952
Epoch 4/40
6/6 [==============================] - 1s 105ms/step - loss: 3.7194 - accuracy: 0.1190
Epoch 5/40
6/6 [==============================] - 1s 101ms/step - loss: 3.2698 - accuracy: 0.2857
Epoch 6/40
6/6 [==============================] - 1s 112ms/step - loss: 3.2125 - accuracy: 0.1905
Epoch 7/40
6/6 [==============================] - 1s 103ms/step - loss: 3.4502 - accuracy: 0.2143
Epoch 8/40
6/6 [==============================] - 1s 114ms/step - loss: 2.9877 - accuracy: 0.2857
Epoch 9/40
6/6 [==============================] - 1s 113ms/step - loss: 2.6633 - accuracy: 0.3571
Epoch 10/40
6/6 [==============================] - 1s 128ms/step - loss: 2.5766 - accuracy: 0.3333
Epoch 11/40
6/

In [11]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.imshow(x_train[3])
plt.show()

plt.figure(figsize=(6,6))
plt.imshow(x_train[6])
plt.show()


# plt.figure(figsize=(6,6))
# plt.imshow(abs(x_train[6]-x_train[3]))
# plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


<Figure size 600x600 with 1 Axes>

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


<Figure size 600x600 with 1 Axes>